In [1]:
import akshare as ak
import pandas as pd
import datetime

In [ ]:
# Global variable - stock market data
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
# Global variable - stock market data filtered
# 总市值 < 200 亿, 0 < 动态市盈率 < 50
stock_market_df_filtered = stock_zh_a_spot_em_df[(stock_zh_a_spot_em_df["总市值"] < 200 * 10e8)
                                                 & (stock_zh_a_spot_em_df["市盈率-动态"] > 0)
                                                 & (stock_zh_a_spot_em_df["市盈率-动态"] < 50)]

  0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
# TODO: Concurrent processing of multiple industries
def industry_info(days=29):
    """
    Extracts the main net flow and change percentage of each industry over the last `days` days.
    :param days: Number of days to consider for net flow calculation.
    :return: DataFrame with industry names, their main net flow, and change percentage.
    """
    # Get the list of industry names
    stock_board_industry_name_em_arr = ak.stock_board_industry_name_em()["板块名称"]
    # TODO: Make the date range dynamic based on the input date
    # Get the first and last date for the specified number of days
    first_date = ak.stock_sector_fund_flow_hist(symbol="证券").iloc[-days]["日期"]
    first_date_str = first_date.strftime("%Y%m%d")
    last_date = ak.stock_sector_fund_flow_hist(symbol="证券").iloc[-1]["日期"]
    last_date_str = last_date.strftime("%Y%m%d")
    # Initialize a pandas Dataframe to hold industry names, industry main net flow, and industry index change percentage
    df = pd.DataFrame(columns=["行业", f"{days}日主力净流入-总净额(亿)", f"{days}日涨跌幅(%)", f"{days}日平均涨跌幅(%)"])
    # Loop through each industry
    for industry_name in stock_board_industry_name_em_arr:
        # Extract industry capital flow data
        stock_sector_fund_flow_hist_df = ak.stock_sector_fund_flow_hist(symbol=industry_name).iloc[-days:]
        # Calculate main net flow
        industry_main_net_flow = stock_sector_fund_flow_hist_df["主力净流入-净额"].sum()
        industry_main_net_flow = round(industry_main_net_flow / 1e8, 2)  # Convert to billions
        # Calcuate change percentage
        stock_board_industry_hist_em = ak.stock_board_industry_hist_em(symbol=industry_name, start_date=first_date_str, end_date=last_date_str, period="日k", adjust="")
        industry_1st_index = stock_board_industry_hist_em["收盘"].iloc[0]
        industry_last_index = stock_board_industry_hist_em["收盘"].iloc[-1]
        industry_index_change_percentage = (industry_last_index - industry_1st_index) / industry_1st_index * 100
        industry_index_change_percentage = round(industry_index_change_percentage, 2)
        # Calcuate avg change percentage
        industry_index_avg_change_percentage = stock_board_industry_hist_em["涨跌幅"].mean()
        industry_index_avg_change_percentage = round(industry_index_avg_change_percentage, 2)
        # print the results
        print(f"{industry_name}: {industry_main_net_flow}, {industry_index_change_percentage}%, {industry_index_avg_change_percentage}%")
        # Append the results to the DataFrame
        df.loc[len(df)] = [industry_name, industry_main_net_flow, industry_index_change_percentage, industry_index_avg_change_percentage]
    # Sort the DataFrame by main net flow in descending order
    df.sort_values(by=f"{days}日主力净流入-总净额(亿)", ascending=False, inplace=True)
    return df

In [18]:
# {days}日主力净流入-总净额 > 50 亿, 并且{days}日平均涨跌幅(%) < 8
days=29
industry_df = industry_info(days=days)
industry_df_filtered = industry_df[(industry_df[f"{days}日主力净流入-总净额(亿)"] > 50) & (industry_df[f"{days}日平均涨跌幅(%)"] < 8)]
industry_df_filtered

0it [00:00, ?it/s]

能源金属: -5.56, 14.03%, 0.48%
贵金属: -40.95, 5.33%, 0.32%
电力行业: -95.04, 6.45%, 0.21%
电机: -32.98, 5.08%, 0.2%
汽车服务: -5.26, 4.92%, 0.17%
电源设备: -58.88, 11.68%, 0.38%
电网设备: -157.98, 7.9%, 0.28%
塑料制品: -86.84, 8.86%, 0.34%
银行: 22.49, 7.33%, 0.32%
公用事业: -10.04, 5.43%, 0.19%
石油行业: -24.27, 7.76%, 0.28%
化学制药: -211.41, 7.53%, 0.38%
通用设备: -226.91, 5.52%, 0.2%
仪器仪表: -33.24, 6.98%, 0.27%
医药商业: -19.39, 4.38%, 0.22%
专业服务: -19.15, 10.5%, 0.37%
交运设备: -51.54, 5.94%, 0.23%
生物制品: -67.16, 5.76%, 0.3%
家电行业: -33.02, 5.64%, 0.2%
汽车零部件: -303.39, 3.58%, 0.14%
纺织服装: -61.62, 4.0%, 0.2%
中药: -64.78, 2.2%, 0.15%
有色金属: -83.63, 11.6%, 0.43%
造纸印刷: -44.62, 6.05%, 0.22%
专用设备: -262.13, 7.63%, 0.28%
橡胶制品: -16.72, 4.59%, 0.18%
工程机械: -35.87, 4.22%, 0.13%
电池: -132.71, 15.37%, 0.53%
化肥行业: -32.7, 8.45%, 0.3%
化学原料: -85.86, 6.23%, 0.24%
化学制品: -156.77, 7.47%, 0.29%
化纤行业: -30.49, 1.7%, 0.07%
环保行业: -98.15, 5.3%, 0.18%
物流行业: -62.9, 2.26%, 0.13%
工程建设: -36.04, 6.14%, 0.22%
电子元件: -143.88, 16.87%, 0.57%
非金属材料: -8.25, 13.05%, 0.46%
农药兽药: -94.79

,行业,29日主力净流入-总净额(亿),29日涨跌幅(%),29日平均涨跌幅(%)
75,保险,59.45,6.05,0.24
78,证券,51.36,13.09,0.47


In [ ]:
# TODO: Concurrent processing of multiple stocks in an industry
def stock_analysis(industry_name, days=29):
    """
    Analyzes stocks in a given industry by extracting their main net inflow and price change percentage over the last `days` days.
    :param industry: The industry name to analyze.
    :param days: Number of days to consider for net flow calculation.
    :return: None, prints the stock name, main net inflow, and price change
    """
    # Extract all stocks from the industry board
    stock_board_industry_cons_em_df = ak.stock_board_industry_cons_em(symbol=industry_name)
    # Extract stock names and codes from the industry board
    stock_info = stock_board_industry_cons_em_df[["代码", "名称"]]

    # Initialize a pandas Dataframe to hold industry names, industry main net flow, and industry index change percentage
    df = pd.DataFrame(columns=["行业", "代码", "名称", "总市值(亿)", "流通市值(亿)", "市盈率-动态", "市净率", f"{days}日主力净流入-总净额(亿)", f"{days}日涨跌幅(%)", f"{days}日平均涨跌幅(%)", "60日涨跌幅(%)", "年初至今涨跌幅(%)"])

    for row in stock_info.itertuples():
        stock_code = row.代码
        stock_name = row.名称
        # Determine the market based on the stock code
        if stock_code.startswith("6"):
            market = "sh"
        elif stock_code.startswith("0") or stock_code.startswith("3"):
            market = "sz"
        else:
            market = "bj"

        # Extract the stock's market data
        stock_total_market_value = stock_zh_a_spot_em_df[stock_zh_a_spot_em_df["代码"] == stock_code]["总市值"].values[0] / 1e8  # Convert to billions
        stock_circulating_market_value = stock_zh_a_spot_em_df[stock_zh_a_spot_em_df["代码"] == stock_code]["流通市值"].values[0] / 1e8  # Convert to billions
        stock_pe_dynamic = stock_zh_a_spot_em_df[stock_zh_a_spot_em_df["代码"] == stock_code]["市盈率-动态"].values[0]
        stock_pb = stock_zh_a_spot_em_df[stock_zh_a_spot_em_df["代码"] == stock_code]["市净率"].values[0]
        stock_60d_change = stock_zh_a_spot_em_df[stock_zh_a_spot_em_df["代码"] == stock_code]["60日涨跌幅"].values[0]
        stock_ytd_change = stock_zh_a_spot_em_df[stock_zh_a_spot_em_df["代码"] == stock_code]["年初至今涨跌幅"].values[0]
        
        # Extract the historical data of the stock
        stock_individual_fund_flow_df = ak.stock_individual_fund_flow(stock=stock_code, market=market).iloc[-days:]
        # Get the main net inflow data
        stock_main_net_flow = stock_individual_fund_flow_df["主力净流入-净额"].sum()
        stock_main_net_flow = round(stock_main_net_flow / 1e8, 2)  # Convert to billions
        # Calculate change percentage
        stock_1st_price = stock_individual_fund_flow_df.iloc[-days]["收盘价"]
        stock_last_price = stock_individual_fund_flow_df.iloc[-1]["收盘价"]
        stock_price_change_percentage = (stock_last_price - stock_1st_price) / stock_1st_price * 100
        stock_price_change_percentage = round(stock_price_change_percentage, 2)
        # Calcuate avg change percentage
        stock_price_avg_change_percentage = stock_individual_fund_flow_df["涨跌幅"].mean()
        stock_price_avg_change_percentage = round(stock_price_avg_change_percentage, 2)
        print(f"{stock_name}: {stock_total_market_value}, {stock_circulating_market_value}, {stock_pe_dynamic}, {stock_pb}, {stock_main_net_flow}, {stock_price_change_percentage}%, {stock_price_avg_change_percentage}%, {stock_60d_change}%, {stock_ytd_change}%")

        # Append the results to the DataFrame
        df.loc[len(df)] = [industry_name, stock_code, stock_name, stock_total_market_value, stock_circulating_market_value, stock_pe_dynamic, stock_pb, stock_main_net_flow, stock_price_change_percentage, stock_price_avg_change_percentage, stock_60d_change, stock_ytd_change]
    # Sort the DataFrame by main net flow in descending order
    df.sort_values(by=f"{days}日主力净流入-总净额(亿)", ascending=False, inplace=True)
    return df

In [12]:
stock_df = pd.DataFrame(columns=["行业", "代码", "名称", "总市值(亿)", "流通市值(亿)", "市盈率-动态", "市净率", f"{days}日主力净流入-总净额(亿)", f"{days}日涨跌幅(%)", f"{days}日平均涨跌幅(%)", f"{days}日平均涨跌幅(%)", "60日涨跌幅(%)", "年初至今涨跌幅(%)"])
for industry_name in industry_df_filtered["行业"]:
    stock_df = stock_analysis(industry_name)
    stock_df = pd.concat([stock_df, stock_df], ignore_index=True)

0it [00:00, ?it/s]

新华保险: 1925.69611618, 1287.34170458, 8.18, 2.41, 3.12, 20.45%, 0.71%, 28.18%, 24.21%
中国人保: 3759.03919956, 3017.30930956, 7.31, 1.35, 4.82, 1.92%, 0.1%, 22.3%, 12.43%
中国平安: 10507.30536824, 6210.05349002, 9.72, 1.14, 30.28, 10.45%, 0.39%, 19.64%, 13.03%
中国人寿: 11404.8084675, 8402.294355, 9.9, 2.14, 4.1, 5.11%, 0.23%, 13.57%, -2.7%
中国太保: 3685.55281141, 2622.33538141, 9.57, 1.4, 17.22, 11.48%, 0.39%, 25.9%, 12.41%
*ST天茂: 103.9677503, 95.7395003, -23.41, 0.48, -2.97, -42.38%, -1.82%, -41.27%, -54.6%


0it [00:00, ?it/s]

国联民生: 670.87801039, 282.15662714, 44.59, 1.33, -0.26, 11.17%, 0.39%, 21.13%, -12.65%
华西证券: 256.9875, 256.9875, 21.34, 1.08, -1.1, 13.81%, 0.5%, 21.31%, 17.81%
中银证券: 372.252, 372.252, 33.19, 2.04, 8.88, 30.92%, 1.0%, 35.49%, 20.07%
国信证券: 1170.79389812, 1113.11093209, 12.57, 1.35, -0.81, 9.92%, 0.34%, 26.35%, 12.26%
长城证券: 361.08121256, 312.14225465, 14.91, 1.18, -1.31, 9.84%, 0.33%, 17.61%, 9.15%
广发证券: 1353.84050096, 1050.92077736, 12.28, 1.12, 0.95, 10.91%, 0.38%, 19.14%, 12.59%
兴业证券: 552.70318682, 552.70318682, 26.76, 1.01, 3.14, 9.43%, 0.38%, 10.73%, 2.24%
华鑫股份: 167.30381835, 167.30381835, 30.44, 1.96, -2.5, 10.63%, 0.42%, 19.92%, -4.31%
招商证券: 1543.63350807, 1317.40593578, 16.72, 1.36, -0.27, 8.9%, 0.33%, 3.2%, -7.36%
XD中国银: 1886.18438916, 1249.48953997, 15.64, 1.68, 0.86, 9.89%, 0.35%, 10.58%, 14.77%
山西证券: 221.48890445, 221.48890445, 22.19, 1.23, -0.36, 11.51%, 0.41%, 9.98%, -0.96%
方正证券: 667.62342313, 667.62342313, 13.97, 1.37, -1.17, 6.42%, 0.23%, 9.89%, -2.64%
华泰证券: 1658.31542902, 

In [ ]:
# 总市值 < 200 亿, 动态市盈率 < 50, {days}日主力净流入-总净额(亿) > 1, {days}日涨跌幅(%) < 10
stock_df_filtered = stock_df[(stock_df["总市值(亿)"] < 200) 
                             & (stock_df["市盈率-动态"] < 50) 
                             & (stock_df[f"{days}日主力净流入-总净额(亿)"] > 1) 
                             & (stock_df[f"{days}日涨跌幅(%)"] < 10)]
stock_df_filtered  

,行业,代码,名称,总市值(亿),流通市值(亿),市盈率-动态,市净率,29日主力净流入-总净额(亿),29日涨跌幅(%),29日平均涨跌幅(%),60日涨跌幅(%),年初至今涨跌幅(%)


In [22]:
stock_df[(stock_df["总市值(亿)"] < 200) 
                             & (stock_df["市盈率-动态"] < 50)]

,行业,代码,名称,总市值(亿),流通市值(亿),市盈率-动态,市净率,29日主力净流入-总净额(亿),29日涨跌幅(%),29日平均涨跌幅(%),60日涨跌幅(%),年初至今涨跌幅(%)
21,证券,600864,哈投股份,137.525711,137.525711,28.98,1.08,-0.05,19.86,0.69,13.77,-9.45
23,证券,000712,锦龙股份,117.376000,117.341170,-30.65,5.19,-0.24,10.52,0.36,15.11,-13.36
26,证券,600155,华创云信,159.153704,159.153704,-100.39,0.81,-0.32,11.59,0.41,7.47,-2.84
44,证券,000686,东北证券,191.449048,191.449048,23.66,1.01,-2.21,10.77,0.42,10.09,4.60
45,证券,600621,华鑫股份,167.303818,167.303818,30.44,1.96,-2.50,10.63,0.42,19.92,-4.31
70,证券,600864,哈投股份,137.525711,137.525711,28.98,1.08,-0.05,19.86,0.69,13.77,-9.45
72,证券,000712,锦龙股份,117.376000,117.341170,-30.65,5.19,-0.24,10.52,0.36,15.11,-13.36
75,证券,600155,华创云信,159.153704,159.153704,-100.39,0.81,-0.32,11.59,0.41,7.47,-2.84
93,证券,000686,东北证券,191.449048,191.449048,23.66,1.01,-2.21,10.77,0.42,10.09,4.60
94,证券,600621,华鑫股份,167.303818,167.303818,30.44,1.96,-2.50,10.63,0.42,19.92,-4.31


In [ ]:
stock_df[(stock_df["总市值(亿)"] < 200) 
                             & (stock_df["市盈率-动态"] < 50)]

In [ ]:
# Define the report date
last_date = ak.stock_sector_fund_flow_hist(symbol="证券").iloc[-1]["日期"]
last_date_str = last_date.strftime("%Y%m%d")
# Output the industry_df_filtered to a CSV file
industry_df_filtered.to_csv(f"reports/行业筛选报告-{last_date_str}.csv", index=True)
# Output the stock_df_filtered to a CSV file
stock_df_filtered.to_csv(f"reports/股票筛选报告-{last_date_str}.csv", index=True)

datetime.date(2025, 7, 11)

In [26]:
ak.index_stock_info()

,index_code,display_name,publish_date
0,000001,上证指数,1991-07-15
1,000002,A股指数,1992-02-21
2,000003,B股指数,1992-02-21
3,000004,工业指数,1993-05-03
4,000005,商业指数,1993-05-03
...,...,...,...
720,399994,中证信息安全主题指数,2015-03-12
721,399995,中证基建工程指数,2015-03-12
722,399996,中证智能家居指数,2014-09-17
723,399997,中证白酒指数,2015-01-21


In [ ]:
import pandas as pd
df = pd.read_csv("reports/股票筛选报告-20231027.csv", index_col=0)

In [ ]:
df

: 